In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch

f:\Archivosdelprograma\Environments\lyrics_gen\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
# Comprobar funcionamiento de CUDA.
torch.cuda.empty_cache()
print("CUDA disponible:", torch.cuda.is_available())
print("Dispositivo actual:", torch.cuda.get_device_name(0))

CUDA disponible: True
Dispositivo actual: NVIDIA GeForce RTX 3060


Función generadora

In [3]:
def generate_lyrics(
    model_checkpoint_path,
    title,
    artist,
    lyrics_start="",
    max_length=150,
    temperature=1.0,
    top_k=50,
    top_p=0.95,
    device="cuda" if torch.cuda.is_available() else "cpu"
):
    # Cargar tokenizer y modelo
    tokenizer = GPT2TokenizerFast.from_pretrained("gpt2-medium")
    tokenizer.add_special_tokens({
        "additional_special_tokens": ["<|endoflyric|>"],
        "pad_token": "<|pad|>"
    })
    tokenizer.pad_token = tokenizer.eos_token

    model = GPT2LMHeadModel.from_pretrained(model_checkpoint_path)
    model.resize_token_embeddings(len(tokenizer))
    model.to(device)
    model.eval()

    # Construir el prompt
    prompt = f"[Artista: {artist}]\n[Canción: {title}]\n{lyrics_start}"
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Generación
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.convert_tokens_to_ids("<|endoflyric|>")
    )

    # Decodificar
    generated = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated


In [5]:
rock_model_path = "./lyrics_generator_rock/checkpoint-1000"

generated_rock_lyrics = generate_lyrics(
    model_checkpoint_path=rock_model_path,
    title="Unintended",
    artist="Muse",
    lyrics_start="You could be my unintended",
    max_length=200
)

print(generated_rock_lyrics)

[Artista: Muse]
[Canción: Unintended]
You could be my unintended baby
I've learned to keep my distance
But I see you're still there

You’ll be my unexpected baby
And you've done it before
We all know that it can't last forever

You’ll be my unexpected baby
And you’ve done it before
We all know that it can't last forever

I can't say it
So I will just say it

I can't say it
So I will just say it
I can't say it
So I will just say it

I'm still going to hurt you
I am, I am
And I’ll stay that way
But we'll get back to the truth
(And I’m still going to hurt you)
And you’ll find a way to turn that around
So I will just
